In [2]:
import pandas as pd

# Lee el archivo .xls
archivo_xls = pd.read_excel('pib_municipios_sector_2020.xls')

# Guarda los datos en un archivo .csv
archivo_xls.iloc[8:].to_csv('pib_municipios_sector_2020.csv', index=False, sep=';')


In [3]:
import pandas as pd
import re

# Cargar los datos del primer CSV
df1 = pd.read_csv('pib_municipios_sector_2020.csv', delimiter=';')
df1.columns = ['Nombre', 'pib_primario', 'pib_secundario_1', 'pib_secundario_2', 'pib_terciario_1', 'pib_terciario_2', 'pib_terciario_3']
df1 = df1.dropna(subset=['Nombre', 'pib_secundario_1', 'pib_secundario_2', 'pib_terciario_1', 'pib_terciario_2', 'pib_terciario_3'], how='all')

# # Cargar los datos del segundo CSV
df2 = pd.read_csv('../union_2020/paro_sector.csv', delimiter=';', dtype={'Codigo_municipio': str})

# Eliminar las columnas no deseadas del segundo DataFrame
df2 = df2.drop(columns=['paro_primario', 'paro_secundario', 'paro_terciario'])

for i, name in df1['Nombre'].items():
    escaped_name = re.escape(str(name))
    matched_rows = df2[df2['Nombre'].str.contains(escaped_name)]
    if not matched_rows.empty:
        df1.loc[i, 'Nombre'] = matched_rows['Nombre'].values[0]

# Combinar los DataFrames usando el nombre como clave
result = pd.merge(df1, df2, on='Nombre', how='inner')
result = result[['Serie', 'Codigo_municipio', 'Nombre', 'pib_primario', 'pib_secundario_1', 'pib_secundario_2', 'pib_terciario_1', 'pib_terciario_2', 'pib_terciario_3']]
# print(result[result['Serie'] == 'Zonas Estadísticas'])
# # Guardar el resultado en un nuevo archivo CSV
result.to_csv('pib_sector.csv', index=False, sep=';')


In [27]:
result = pd.read_csv('pib_sector.csv', delimiter=';', dtype={'Codigo_municipio': str})

result['pib_secundario'] = result['pib_secundario_1'] + result['pib_secundario_2']
result['pib_terciario'] = result['pib_terciario_1'] + result['pib_terciario_2'] + result['pib_terciario_3']

# Agregar una columna 'sector' basada en el sector predominante
for index, row in result.iterrows():
    pib_primario = row['pib_primario']
    pib_secundario = row['pib_secundario']
    pib_terciario = row['pib_terciario']
    
    if pib_primario > pib_secundario and pib_primario > pib_terciario:
        result.loc[index, 'sector'] = 'Primario'
    elif pib_secundario > pib_primario and pib_secundario > pib_terciario:
        result.loc[index, 'sector'] = 'Secundario'
    else:
        result.loc[index, 'sector'] = 'Terciario'

result = result.iloc[:, [0, 1, 2, -1]]

# Mostrar el resultado
result.to_csv('../union_2020/sector.csv', index=False, sep=';')
